# ROOT_SOLVER

## Overview
Solves systems of nonlinear equations by finding values for variables that satisfy all equations simultaneously. This function enables business users to solve complex, highly nonlinear relationships directly in Excel, leveraging the [SciPy root method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html) for advanced root-finding. It is suitable for square systems (number of equations equals number of variables).

A system of nonlinear equations can be written as:

```math
F(\mathbf{x}) = \begin{bmatrix} f_1(x_1, x_2, ..., x_n) \\ f_2(x_1, x_2, ..., x_n) \\ \vdots \\ f_n(x_1, x_2, ..., x_n) \end{bmatrix} = \mathbf{0}
```

where $\mathbf{x} = [x_1, x_2, ..., x_n]$ is the vector of variables, and each $f_i$ is a nonlinear function. The goal is to find $\mathbf{x}^*$ such that all equations are satisfied simultaneously.

This function uses SciPy's `scipy.optimize.root` for root-finding. The process continues until $\|F(\mathbf{x}_k)\|$ is below a specified tolerance or a maximum number of iterations is reached.

See the [scipy.optimize.root documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html) for more details.

This example function is provided as-is without any representation of accuracy.

## Usage
To use the `ROOT_SOLVER` function in Excel, enter it as a formula in a cell, specifying your equations and initial guesses:

```excel
=ROOT_SOLVER(equations, variables)
```

- `equations` (2D list of string, required): Symbolic expressions for the system of equations to solve. Example: `[ ["x**2 + y**2 - 1", "x - y"] ]`
- `variables` (2D list of float, required): Initial guesses for the variables. Example: `[[0.5, 0.5]]`

The function returns a 2D list of floats representing the solution for the variables that satisfy the system, or a string error message if the solver fails or input is invalid.

## Examples

**Example 1: Chemical Equilibrium**

A chemist needs to solve for concentrations of two substances at equilibrium, given two nonlinear equations representing the chemical reactions.

In Excel:
```excel
=ROOT_SOLVER({"x**2 + y**2 - 1", "x - y"}, {0.5, 0.5})
```
Expected output:

| x      | y      |
|--------|--------|
| 0.7071 | 0.7071 |

**Example 2: Engineering Circuit Analysis**

An engineer wants to find the voltages in a nonlinear circuit described by two equations.

In Excel:
```excel
=ROOT_SOLVER({"x**3 + y - 1", "x + y**3 - 1"}, {0.7, 0.7})
```
Expected output:

| x      | y      |
|--------|--------|
| 0.6826 | 0.6826 |

In [ ]:
import numpy as np
import re
from scipy.optimize import root


def root_solver(equations, variables):
    """
    Solves a square system of nonlinear equations using SciPy's root-finding capabilities.

    Args:
        equations (list[list[str]]): 2D list of symbolic expressions for the system (e.g., [["x**2 + y**2 - 1"], ["x - y"]] or [["x**2 + y**2 - 1", "x - y"]]).
        variables (list[list[float]]): 2D list of initial guesses for the variables (e.g., [[0.5, 0.5]]).

    Returns:
        list[list[float]]: The solution for the variables that satisfy the system, or
        str: Error message if the solver fails or input is invalid.

    This example function is provided as-is without any representation of accuracy.
    """
    if not (isinstance(equations, list) and len(equations) > 0 and all(isinstance(row, list) for row in equations)):
        return "equations must be a 2D list of strings."
    if not (isinstance(variables, list) and len(variables) > 0 and isinstance(variables[0], list)):
        return "variables must be a 2D list of floats."
    # Flatten all rows of equations (works for row or column vector)
    eqn_list = [eq for row in equations for eq in row]
    n_eqns = len(eqn_list)
    var_vals = variables[0]
    n_vars = len(var_vals)
    if n_vars != n_eqns:
        return f"Error: number of variables ({n_vars}) must match number of equations ({n_eqns})."
    var_names = [chr(ord('x') + i) for i in range(n_vars)]
    def substitute_vars(expr, var_names):
        for i, name in enumerate(var_names):
            expr = re.sub(rf'(?<![A-Za-z0-9_]){name}(?![A-Za-z0-9_])', f'x[{i}]', expr)
        return expr
    def fun(x):
        local_dict = {'x': x}
        f_list = []
        for eq in eqn_list:
            try:
                eq_sub = substitute_vars(eq, var_names)
                f_list.append(eval(eq_sub, local_dict))
            except Exception as e:
                return [float('nan')]*n_eqns
        return f_list
    try:
        sol = root(fun, np.array(var_vals, dtype=float))
    except Exception as e:
        return f"Solver failed: {str(e)}"
    if not sol.success:
        return f"Solver failed: {sol.message}"
    return [sol.x.flatten().tolist()]

In [ ]:
import ipytest
ipytest.autoconfig()
import pytest

demo_cases = [
    [ [ ["x**2 + y**2 - 1"], ["x - y"] ], [ [0.5, 0.5] ], [[0.70710678, 0.70710678]]],
    [ [ ["x**3 + y - 1"], ["x + y**3 - 1"] ], [ [0.7, 0.7] ], [[0.68260619, 0.68260619]]],
]

def approx_equal(a, b, rel=0.05, abs_tol=1e-4):
    if isinstance(a, float) and isinstance(b, float):
        return a == pytest.approx(b, rel=rel, abs=abs_tol)
    if (
        isinstance(a, list) and isinstance(b, list)
        and all(isinstance(x, list) for x in a)
        and all(isinstance(y, list) for y in b)
    ):
        return all(
            all(isinstance(x, float) and isinstance(y, float) and x == pytest.approx(y, rel=rel, abs=abs_tol) for x, y in zip(row_a, row_b))
            for row_a, row_b in zip(a, b)
        )
    return False

@pytest.mark.parametrize("equations, variables, expected", demo_cases)
def test_demo_cases(equations, variables, expected):
    # Excel always outputs a 2D list, so no need to wrap equations
    result = root_solver(equations, variables)
    if isinstance(result, list):
        assert approx_equal(result, expected, rel=0.05)
    else:
        assert isinstance(result, str)

def test_invalid_equation():
    result = root_solver([["x**2 +"]], [[0.5, 0.5]])
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

def test_mismatched_vars():
    result = root_solver([["x**2 + y**2 - 1"], ["x - y"]], [[0.5]])
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

ipytest.run('-s')

In [ ]:
# Gradio Demo (with lowercase headers)
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

def get_var_headers(variables):
    # Always return ['x', 'y', 'z'] up to the number of variables
    try:
        n_vars = len(variables[0])
        return [c for c in 'xyz'[:n_vars]]
    except Exception:
        return []

def gradio_root_solver(equations, variables):
    result = root_solver(equations, variables)
    headers = get_var_headers(variables)
    # Prepare result table (DataFrame)
    if isinstance(result, str):
        result_df = [[result] + [None]*(len(headers)-1)] if headers else [[result]]
        img_html = ""
    else:
        result_df = result
        # Pad result rows if needed
        if headers and len(result_df[0]) < len(headers):
            result_df = [row + [None]*(len(headers)-len(row)) for row in result_df]
        # Plot for 2 variables and 2 equations only
        try:
            eqn_list = [eq for row in equations for eq in row]
            n_eqns = len(eqn_list)
            var_vals = variables[0]
            n_vars = len(var_vals)
            if n_vars == 2 and n_eqns == 2:
                var_names = [c for c in 'xy']
                x_range = np.linspace(-2, 2, 400)
                y_range = np.linspace(-2, 2, 400)
                X, Y = np.meshgrid(x_range, y_range)
                fig, ax = plt.subplots(figsize=(5, 5))
                for i, eq in enumerate(eqn_list):
                    expr = eq
                    expr = expr.replace(var_names[0], 'X').replace(var_names[1], 'Y')
                    try:
                        Z = eval(expr, {"X": X, "Y": Y, "np": np, "math": np})
                        ax.contour(X, Y, Z, levels=[0], colors=["C%d" % i], linewidths=2)
                    except Exception:
                        continue
                if isinstance(result, list) and len(result) > 0 and isinstance(result[0], list) and len(result[0]) == 2:
                    sol_x, sol_y = result[0]
                    ax.plot([sol_x], [sol_y], 'ro', label="Root")
                    ax.annotate(f"({sol_x:.3g}, {sol_y:.3g})", (sol_x, sol_y), textcoords="offset points", xytext=(0,10), ha='center', color='red')
                ax.set_xlabel('x')
                ax.set_ylabel('y')
                ax.set_title("System of Equations")
                ax.legend()
                buf = io.BytesIO()
                plt.tight_layout()
                plt.savefig(buf, format="png")
                plt.close()
                buf.seek(0)
                img_base64 = base64.b64encode(buf.read()).decode("utf-8")
                img_html = f'<img src="data:image/png;base64,{img_base64}" style="max-width:100%;height:auto;" />'
            else:
                img_html = ""
        except Exception:
            img_html = ""
    return result_df, img_html

# Set headers for variables and roots as x, y, z (or fewer as needed)
var_headers = get_var_headers(demo_cases[0][1])
demo = gr.Interface(
    fn=gradio_root_solver,
    inputs=[
        gr.DataFrame(label="Equations", value=demo_cases[0][0], type="array", headers=["Equations"]),
        gr.DataFrame(label="Variables", value=demo_cases[0][1], type="array", headers=var_headers),
    ],
    outputs=[
        gr.DataFrame(label="Roots", type="array", headers=var_headers),
        gr.HTML(label="Plot")
    ],
    examples=demo_cases,
    flagging_mode="never",
    fill_width=True,
)
demo.launch()